In [48]:
import numpy as np
import pandas as pd
from ModelTester import Models
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [6]:
data = pd.read_csv('clean_train_data.csv').drop(['ID','Credit_Score'], axis = 1).dropna()
models = Models(data)
models.split_data()

In [2]:
#models.models_individual()

c:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Modelo: Random Forest Classifier
Accuracy en Train: 1.0000
Accuracy en Test: 0.7708
----------------------------------------
Modelo: HGBC
Accuracy en Train: 0.7341
Accuracy en Test: 0.7002
----------------------------------------
Modelo: K-Nearest Neighbors (weighted by distance)
Accuracy en Train: 1.0000
Accuracy en Test: 0.6967
----------------------------------------
Modelo: Decision Tree Classifier
Accuracy en Train: 1.0000
Accuracy en Test: 0.6891
----------------------------------------
Modelo: Gradient Boosting Classifier
Accuracy en Train: 0.6773
Accuracy en Test: 0.6673
----------------------------------------
Modelo: K-Nearest Neighbors (uniformly weighted)
Accuracy en Train: 0.7842
Accuracy en Test: 0.6625
----------------------------------------
Modelo: Multilayer Perceptron
Accuracy en Train: 0.6281
Accuracy en Test: 0.6253
----------------------------------------
Modelo: Linear Discriminant Analysis
Accuracy en Train: 0.6050
Accuracy en Test: 0.6021
----------------------

In [9]:
X_train = models.X_train
X_test = models.X_test
y_train = models.y_train
y_test = models.y_test

KNC_d = KNeighborsClassifier(weights='distance').fit(X_train, y_train)
DTC = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
RFC = RandomForestClassifier(random_state=0).fit(X_train, y_train)
HGBC = HistGradientBoostingClassifier().fit(X_train, y_train)

mds = {'K-Nearest Neighbors (weighted by distance)':KNC_d,
           'Decision Tree Classifier':DTC,
          'Random Forest Classifier':RFC,
          'HGBC':HGBC,
         }

# Crear diccionarios para almacenar los resultados de train y test
train_accuracies = {}
test_accuracies = {}

# Calcular el accuracy para cada modelo en los conjuntos de train y test
for model_name, model in mds.items():
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    train_accuracies[model_name] = train_accuracy
    test_accuracies[model_name] = test_accuracy

# Ordenar los modelos por accuracy en el conjunto de prueba en orden descendente
sorted_models_test = {k: v for k, v in sorted(test_accuracies.items(), key=lambda item: item[1], reverse=True)}

# Imprimir los resultados ordenados
for model_name, test_accuracy in sorted_models_test.items():
    train_accuracy = train_accuracies[model_name]
    print(f'Modelo: {model_name}')
    print(f'Accuracy en Train: {train_accuracy:.4f}')
    print(f'Accuracy en Test: {test_accuracy:.4f}')
    print('-' * 40)

Modelo: Random Forest Classifier
Accuracy en Train: 1.0000
Accuracy en Test: 0.7708
----------------------------------------
Modelo: HGBC
Accuracy en Train: 0.7342
Accuracy en Test: 0.7032
----------------------------------------
Modelo: K-Nearest Neighbors (weighted by distance)
Accuracy en Train: 1.0000
Accuracy en Test: 0.6967
----------------------------------------
Modelo: Decision Tree Classifier
Accuracy en Train: 1.0000
Accuracy en Test: 0.6891
----------------------------------------


Accuracy en datos de entrenamiento: 0.9999869451697128
Accuracy en datos de prueba: 0.7708213670304422


KNeighborsClassifier(weights='distance')

In [30]:
def stacking_manual(KNC_d, DTC, RFC, HGBC, X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame):
    # 1. Realizar predicciones de los modelos base en los datos de entrenamiento y prueba
    mds = [RFC, HGBC]
    train_predictions = []
    test_predictions = []

    for model in mds:
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        train_predictions.append(train_pred)
        test_predictions.append(test_pred)

    # 2. Apilar las predicciones de los modelos base en un nuevo conjunto de datos
    stacked_train = np.column_stack(train_predictions)
    stacked_test = np.column_stack(test_predictions)

    # 3. Entrenar un modelo final (meta-modelo)
    meta_model = KNeighborsClassifier(weights='distance')
    meta_model.fit(stacked_train, y_train)

    # Calcular el accuracy en los datos de entrenamiento y prueba
    train_accuracy = accuracy_score(y_train, meta_model.predict(stacked_train))
    test_accuracy = accuracy_score(y_test, meta_model.predict(stacked_test))

    print("Accuracy en datos de entrenamiento:", train_accuracy)
    print("Accuracy en datos de prueba:", test_accuracy)

    return meta_model

In [ ]:
estimator = [
    ('fr', RandomForestClassifier()),
    ('knn', KNeighborsClassifier(weights='distance')),
    ('',)
]
modelStacking = StackingClassifier(estimator, final_estimator=)

In [39]:
def BaggingManual(model, X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame):
    GBC_bag = BaggingClassifier( model
    ).fit(X_train, y_train)
    y_pred_bag = GBC_bag.predict(X_test)
    print(classification_report(y_test, y_pred_bag))
    return GBC_bag

In [40]:
meta_model = stacking_manual(KNC_d, DTC, RFC, HGBC, X_train, y_train, X_test, y_test)

Accuracy en datos de entrenamiento: 0.9999869451697128
Accuracy en datos de prueba: 0.7708213670304422


In [41]:
BaggingManual(meta_model, X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           1       0.71      0.70      0.71      5595
           2       0.71      0.75      0.73     10157
           3       0.58      0.50      0.54      3399

    accuracy                           0.69     19151
   macro avg       0.67      0.65      0.66     19151
weighted avg       0.69      0.69      0.69     19151



BaggingClassifier(estimator=KNeighborsClassifier(weights='distance'))

In [42]:
BaggingManual(RandomForestClassifier(), X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           1       0.77      0.76      0.77      5595
           2       0.75      0.82      0.78     10157
           3       0.73      0.56      0.64      3399

    accuracy                           0.76     19151
   macro avg       0.75      0.71      0.73     19151
weighted avg       0.75      0.76      0.75     19151



BaggingClassifier(estimator=RandomForestClassifier())

In [47]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# Create an instance of SMOTE
smote = SMOTE(random_state=42)

# Resample the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize the Random Forest Classifier with bootstrap enabled
rf_clf = RandomForestClassifier(bootstrap=True, random_state=42)

# Fit the model on the resampled training data
rf_clf.fit(X_train_resampled, y_train_resampled)

# Make predictions on the resampled training set and the original test set
train_preds = rf_clf.predict(X_train_resampled)
test_preds = rf_clf.predict(X_test)

# Calculate confusion matrices
train_confusion_matrix = confusion_matrix(y_train_resampled, train_preds)
test_confusion_matrix = confusion_matrix(y_test, test_preds)

# Calculate other metrics
train_accuracy = accuracy_score(y_train_resampled, train_preds)
test_accuracy = accuracy_score(y_test, test_preds)

train_precision = precision_score(y_train_resampled, train_preds, average='weighted')
test_precision = precision_score(y_test, test_preds, average='weighted')

train_recall = recall_score(y_train_resampled, train_preds, average='weighted')
test_recall = recall_score(y_test, test_preds, average='weighted')

train_f1 = f1_score(y_train_resampled, train_preds, average='weighted')
test_f1 = f1_score(y_test, test_preds, average='weighted')

# Print the metrics
print("Train Confusion Matrix:\n", train_confusion_matrix)
print("Test Confusion Matrix:\n", test_confusion_matrix)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

print("Train Precision:", train_precision)
print("Test Precision:", test_precision)

print("Train Recall:", train_recall)
print("Test Recall:", test_recall)

print("Train F1 Score:", train_f1)
print("Test F1 Score:", test_f1)


Train Confusion Matrix:
 [[40688     0     0]
 [    1 40687     0]
 [    0     0 40688]]
Test Confusion Matrix:
 [[4545  929  121]
 [1431 7755  971]
 [  44  956 2399]]
Train Accuracy: 0.9999918075763534
Test Accuracy: 0.7675317215811185
Train Precision: 0.9999918077776959
Test Precision: 0.7691930541854481
Train Recall: 0.9999918075763534
Test Recall: 0.7675317215811185
Train F1 Score: 0.9999918075763522
Test F1 Score: 0.7677502434655393
